# LSTM in TensorFlow: IMDb Sentiment Classification

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras import layers, Input, Model

Unter MacOS mit Apple Silicon-Prozessor (M1, M2, M3) funktionieren LSTM und andere rekurrente Netze leider noch immer nicht, wenn sie mit Maskierung kombiniert werden. Daher müssen wir auf dieser Hardware-Platform die GPU deaktivieren und nur mit der CPU trainieren.

In [ ]:
import platform
if platform.system() == "Darwin" and platform.processor() == "arm":
    print("Disabling GPU on MacOS M1/M2/M3 platform.")
    tf.config.experimental.set_visible_devices([], 'GPU')

## Datensatz & Vorverarbeitung

In dieser Übung beschäftigen wir uns zum ersten Mal mit rekurrenten neuronalen Netzen (insb. LSTM) für sequentielle Daten am Beispiel der Textklassifikation. Ein sehr bekannter Datensatz zur Sentimentanalyse von [IMDb movie reviews](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data) ist bereits in TensorFlow enthalten. 

In [ ]:
(text_train, y_train), (text_test, y_test) = imdb.load_data(num_words=10000, seed=4242)
print(len(text_train), 'training items')
print(len(text_test), 'test items')

Trainings- und Testdaten umfassen jeweils 25.000 Rezensionen, die als _positiv_ (1) oder _negativ_ (0) klassifiziert sind (was leider in der Dokumentation nicht erwähnt wird).

In [ ]:
print(np.vstack(np.unique(y_train, return_counts=True)))

Die Texte sind bereits in der für Deep Learning benötigten Form aufbereitet und liegen jeweils als Listen von ganzzahligen Lexikon-IDs vor (auch wenn die Dokumentation leider von _unique tokens_ statt von _types_ spricht). Wir haben beim Laden des Datensatzes bereits das Vokabular auf die 10.000 häufigsten Wörter eingeschränkt. In einer realen Anwendung müssten wir natürlich die Texte zunächst tokenisieren und in diese numerische Darstellung überführen, was am bequemsten mit einem [TextVectorization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization)-Layer erreicht werden kann.

In [ ]:
print(text_train[0])

Die Lexikon-IDs sind nach Häufigkeit vergeben, wobei die ersten Werte für spezielle Symbole reserviert sind: `0` für Padding (worauf wir etwas später zurückkommen), `1` als Markierung für den Anfang des Textes, sowie `2` für OOV (_out of vocabulary_, also Typen, die nicht zu den 10.000 häufigsten Wörtern zählen).  Die eigentlichen Wort-Typen fangen beim Index `4` an (auch wenn die Dokumentation hier etwas irreführend formuliert ist). Die ID `42` steht also für den Wort-Typ auf Häufigkeitsrang 39.

Um die eigentlichen Texte rekonstruieren zu können, benötigen wir Hilfsfunktionen, die Lexikon-IDs auf die entsprechenden Wörter abbilden bzw. umgekehrt.  Eine Wortliste ist bereits im Datensatz enthalten, die dort verzeichneten IDs müssen aber verschoben werden, um Platz für die speziellen Symbole zu schaffen.

In [ ]:
word2id = imdb.get_word_index()
id2word = [""] * (len(word2id) + 4)
for w in word2id:
    i = word2id[w] + 3
    id2word[i] = w
    word2id[w] = i
id2word[0] = "-PAD-"
id2word[1] = "-START-"
id2word[2] = "-OOV-"

Nun können wir die beiden Hilfsfunktionen zur Kodierung und Dekodierung von (bereits tokenisierten) Texten definieren:

In [ ]:
def encode_sent(tokens):
    return [1] + [word2id.get(t, 2) for t in tokens]

def decode_sent(ids, join=False):
    tokens = [id2word[i] for i in ids]
    if join:
        return " ".join(tokens)
    else:
        return tokens

Hier noch einmal die erste Rezension aus den Trainingsdaten als lesbarer Text:

In [ ]:
print(decode_sent(text_train[0], join=True))

Die Textlängen der Rezensionen weisen eine recht schiefe Verteilung auf. Die meisten Texte umfassen wenige hundert Token, während die längsten Rezensionen über 2000 Token lang sind.

In [ ]:
plt.hist([[len(x) for x in text_train], 
          [len(x) for x in text_test]], 
         bins=20, rwidth=1)
plt.legend(("Train", "Test"));

Neuronale Netze erwarten die Eingabedaten für jeden Batch in einem einzigen großen Tensor, nicht als Liste von Listen. Diese Repräsentation ist aber nur möglich, wenn alle Texte im Batch gleich viele Token umfassen. Deshalb müssen zu lange Texte abgeschnitten (`pre` = vorne oder `post` = hinten) und kürzere Texte mit Padding-Token aufgefüllt werden. Dies könnte separate für jeden Batch durchgeführt werden, um die gemeinsame Länge jeweils anzupassen (was besonders sinnvoll ist, wenn die Daten in Batches aus Texten ähnlicher Länge vorgruppiert sind). Der Einfachheit halber führen wir die Anpassung aber als Vorverarbeitung für die kompletten Trainings- und Testdaten durch.

> **Frage:** Was könnte eine sinnvolle Länge sein? Sollten längere Texte vorne oder hinten abgeschnitten werden? Und wo sollte ggf. erforderliches Padding eingefügt werden?

Wir schneiden hier Texte bei 250 Token ab, so dass ein erheblicher Teil der Rezensionen ungekürzt verarbeitet werden kann. Ob vorne oder hinten gekürzt werden sollte ist nicht offensichtlich: es scheint aber üblich, die Gesamteinschätzung gleich am Anfang der Rezension zu vermitteln und sich dann den Einzelheiten zuzuwenden. Daher behalten wir stets den Anfang der Texte.

Klarer ist, wo Padding eingefügt werden sollte: Wir wollen ja ein LSTM anwenden, um Informationen aus den ganzen Texten aufzusammeln und schließlich für die Sentimentanalyse zu nutzen. Würde das Padding hinten eingefügt, so müsste sich das LSTM die gewonnenen Information über alle Padding-Token hinweg merken (während es bei Padding-Token am Anfang der Sequenz einfach in seinem Initialisierungszustand bleiben ann).

In [ ]:
X_train = pad_sequences(text_train, maxlen=250, 
                        truncating="post", padding="pre")
X_test = pad_sequences(text_test, maxlen=250, 
                       truncating="post", padding="pre")
X_train.shape

`X_train` und `X_test` sind $25000 \times 250$-Matrizen. Wir sehen uns noch einmal die erste Rezension aus den Trainingsdaten mit Padding an.

In [ ]:
print(X_train[0, :])
print(decode_sent(X_train[0, :], join=True))

Solche numerischen Textkodierungen können mit einem [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)-Layer in eine subsymbolische Repräsentation (d.h. in Vektoren $\mathbf{x} \in \mathbb{R}^d$) übersetzt werden.  Ein solcher Embedding-Layer ist vollständig äquivalent zu einem Dense-Layer über Eingabevektoren in One-Hot-Kodierung, kann aber wesentlich effizienter implementiert werden.

### Hilfsfunktionen

Wir übernehmen zwei Hilfsfunktionen aus früheren Übungen. Beachten Sie, dass die oben erstellen Trainings- und Testdaten in `train_and_eval()` als Default-Werte hinterlegt werden.

In [ ]:
def history_plot(history, ylim=(0.7, 1.0)):
    plt.figure(figsize=(8, 4))
    xvals = [x + 1 for x in history.epoch]
    plt.plot(xvals, history.history["accuracy"], linewidth=3)
    plt.plot(xvals, history.history["val_accuracy"], linewidth=3)
    plt.axis((1, max(xvals)) + ylim)
    plt.grid(True, axis="y")
    plt.title("Learning curves")
    plt.xlabel("training epoch")
    plt.ylabel("accuracy")
    plt.legend(["Train", "Test"], loc="lower right")

In [ ]:
def train_and_eval(inputs, outputs, epochs=10, batch_size=1024, 
                   X=X_train, y=y_train, test_X=X_test, test_y = y_test,
                   verbose=1, plot=False, summary=False):
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', metrics=['accuracy'])
    if summary:
        print(model.summary())
    
    res = model.fit(X, y, epochs=epochs, batch_size=batch_size, 
                    verbose=verbose, validation_data=(test_X, test_y))

    test_loss, test_acc = model.evaluate(test_X, test_y, batch_size=batch_size)
    print(f'Test accuracy: {100 * test_acc:.3f}%')

    if plot:
        history_plot(res)
    
    return model

## LSTM auf Word-Embeddings

Das einfachste rekurrente Netzwerk für Textklassifikation besteht aus einem Embedding-Layer, einem rekurrenten [LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)-Layer zur Kodierung der kompletten Rezension in einen einzigen Vektor, sowie einem logistischen Layer mit einem einzigen Neuron für die Klassifikationsentscheidung.

Das Format des Eingabe-Layers zeigt an, dass die Textlänge (`None`) nicht vorab festgelegt ist, sondern in jedem Batch unterschiedlich sein kann.

In [ ]:
inputs = Input(shape=(None,))

Für den Embedding-Layer müssen wir die Größe $n_w$ des Vokabulars (so viele Vektorrepräsentationen muss der Layer lernen) sowie die Dimensionalität $\mathbb{R}^d$ der Embedding-Vektoren (hier $d = 100$) angeben. Wir haben $n_w$ zwar beim Laden des Datensatzes festgelegt, am sichersten ist es aber, den korrekten Wert hier noch einmal zu ermitteln. Wir geben hier `mask_zero=True` an, damit die Padding-Tokens beim Training des Modells nicht berücksichtigt werden.

In [ ]:
n_w = X_train.max() + 1
embeddings = layers.Embedding(n_w, 100, mask_zero=True)

Beim LSTM-Layer muss die Anzahl der Neuronen spezifiziert werden (für Ausgabe und den Hidden State). Zahlreiche weitere Optionen sind in der [Dokumentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM) zu finden. Wird für jedes Token eine Ausgabe benötigt (um z.B. weitere LSTM-Layer anzuschließen), so muss auch die Option `return_sequences=True` angegeben werden.

In [ ]:
lstm = layers.LSTM(50)

Der Ausgabelayer benötigt nur ein einzelnes Neuron für die binäre Klassifikation _positiv_ vs. _negativ_. Mit einer logistischen Aktivierungsfunktion kann die Ausgabe als Wahrscheinlichkeit der positiven Klasse (1) interpretiert werden.

In [ ]:
classifier = layers.Dense(1, activation="sigmoid")

Nun können wir die Layer verketten und so das vollständige neuronale Netz erstellen.

> **Frage:** Der Embedding-Layer umfasst 1 Million Parameter ($n_w = 10000 \times d = 100$). Können Sie erklären, wie die 30,200 Parameter des LSTM-Layer zustandekommen?

In [ ]:
outputs = classifier(lstm(embeddings(inputs)))
model = Model(inputs=inputs, outputs=outputs)
model.summary()       

Als Optimierungsverfahren verwenden wir wie immer Adam. Eine geeignete Loss-Funktion ist die [binary cross-entropy](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/binary_crossentropy), welche die durchschnittliche Wahrscheinlichkeit der korrekten Klasse in Bits berechnet.

In [ ]:
model.compile(loss="binary_crossentropy", 
              optimizer="Adam", metrics=["accuracy"])

Wir trainieren unser Modell zunächst einmal für 10 Epochen.

> **Frage:** Denken Sie, dass ein längeres Training sinnvoll wäre?

In [ ]:
history = model.fit(X_train, y_train, batch_size=1024,
                    epochs=10, verbose=1,
                    validation_data=[X_test, y_test])

In [ ]:
history_plot(history)

## Übung

> Versuchen Sie die Sentimentklassifikation zu verbessern, indem Sie mit unterschiedlichen Netzwerktopologien experimentieren und die Hyperparameter optimieren (z.B. Regularisierung). Können Sie bessere Ergebnisse erzielen als unser erstes LSTM? Oder können Sie ähnliche Ergebnisse mit einem einfacheren Netzwerk erreichen, das schneller trainiert werden kann?
> 
> Sie können beispielsweise folgende Dinge ausprobieren:
>  - Verändern Sie die Größe des LSTM-Layers (d.h. die Anzahl der Neuronen).
>  - Reduzieren Sie die Überanpassung durch Regularisierung, z.B. mit einem Dropout-Layer oder dem internen Dropout des LSTM-Layers.
>  - Verändern Sie die Dimensionalität $d$ der Embeddings und die Vokabulargröße $n_w$. Vielleicht wäre es auch besser, nicht einfach die $n_w$ häufigsten Wörter zu verwenden?
>  - Ist es hilfreich, ein tieferes Netzwerk mit mehr Layern zu verwenden? Sie können dazu sowohl LSTM-Layer als auch herkömmliche Layer stapeln. Wenn Sie herkömmliche Dense-Layer auf Tokenebene verwenden wollen, müssen Sie diese per [TimeDistributed](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TimeDistributed) über alle Zeitschritte verteilen.
>  - Was passiert, wenn Sie den LSTM-Layer durch ein [GRU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU) oder ein [SimpleRNN](https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNN) ersetzen?  Sie können den rekurrenten Layer natürlich auch [bidirektional](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional) machen.
>  - Sie könnten auch die Ausgaben des LSTM-Layers aus allen Zeitschritten aufsammeln (z.B. mit [MaxPooling1D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPooling1D)) statt nur den letzten Zustand zu verwenden (oder zusätzlich dazu). So muss das LSTM vielleicht nicht erst lernen, relevante Informationen bis zum letzten Zeitschritt durchzureichen.
>  - Textklassifikation lässt sich oft auch schon mit einem „bag of words“-Ansatz erstaunlich gut durchführen, braucht also vielleicht gar keine rekurrenten Layer. Können Sie in TensorFlow auch einen „bag of words“ oder „bag of embeddings“ implementieren?
>  - Bei einem „bag of words“-Modell sind oft zusätzlich Bigramme und Trigramme wichtig, um eine gute Klassifikation zu erreichen (z.B. muss _nicht schlecht_ ganz anders gewertet werden als _schlecht_). Versuchen Sie, solche N-Gramme mit Hilfe eines Convolution-Layers ([Conv1D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D)) zu berücksichtigen.
>  - Vielleicht können Sie auch vortrainierte Word-Embeddings als Initialisierung verwenden, um eine größeres Vokabular abzudecken und unbekannte Wörter in den Testdaten besser verarbeiten zu können? Geeignete FastText-Embeddings mit $d=100$ finden Sie in der Datei `imdb_embeddings.txt`. 